# Load data, Split into train and test

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
data_2 = pd.read_csv('/content/data-2.csv')

In [30]:
data_2.head()

,Unnamed: 0,$schema,meta,id,type,namespace,title,comment,timestamp,user,log_id,log_type,log_action,log_params,log_action_comment,server_url,server_name,server_script_path,wiki,parsedcomment,minor,patrolled,length,revision,bot
0,0,/mediawiki/recentchange/1.0.0,{'uri': 'https://en.wikipedia.org/wiki/User:41...,1.430187e+09,log,2,User:41.155.9.82,{{blocked p2p proxy|ip=41.155.9.82|isp=Fixed-B...,1633526855,ST47ProxyBot,122426220.0,block,block,"{'duration': '71 hours', 'flags': 'nocreate', ...",blocked User:41.155.9.82 (account creation blo...,https://en.wikipedia.org,en.wikipedia.org,/w,enwiki,{{blocked p2p proxy|ip=41.155.9.82|isp=Fixed-B...,NaN,NaN,NaN,NaN,True
1,1,/mediawiki/recentchange/1.0.0,{'uri': 'https://www.wikidata.org/wiki/Q174788...,1.556013e+09,edit,0,Q17478814,/* wbsetsitelink-add:1|commonswiki */ Category...,1633526854,Deanlaw,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org,www.wikidata.org,/w,wikidatawiki,"‎<span dir=""auto""><span class=""autocomment"">Ad...",False,True,"{'old': 18319, 'new': 18449}","{'old': 1447448374, 'new': 1508596930}",False
2,2,/mediawiki/recentchange/1.0.0,{'uri': 'https://en.wikipedia.org/wiki/User:10...,1.430187e+09,log,2,User:103.108.61.85,{{blocked p2p proxy|ip=103.108.61.85|isp=INCOM...,1633526854,ST47ProxyBot,122426218.0,block,block,"{'duration': '71 hours', 'flags': 'nocreate', ...",blocked User:103.108.61.85 (account creation b...,https://en.wikipedia.org,en.wikipedia.org,/w,enwiki,{{blocked p2p proxy|ip=103.108.61.85|isp=INCOM...,NaN,NaN,NaN,NaN,True
3,3,/mediawiki/recentchange/1.0.0,{'uri': 'https://www.wikidata.org/wiki/Q350193...,1.556013e+09,edit,0,Q3501934,/* wbsetdescription-add:1|yo */ agbábọ́ọ̀lù ọm...,1633526854,T Cells,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org,www.wikidata.org,/w,wikidatawiki,"‎<span dir=""auto""><span class=""autocomment"">Ad...",False,True,"{'old': 26326, 'new': 26436}","{'old': 1508596926, 'new': 1508596929}",False
4,4,/mediawiki/recentchange/1.0.0,{'uri': 'https://www.wikidata.org/wiki/Q384910...,1.556013e+09,edit,0,Q38491095,/* wbeditentity-update-languages-short:0||nl *...,1633526854,Edoderoobot,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org,www.wikidata.org,/w,wikidatawiki,"‎<span dir=""auto""><span class=""autocomment"">Ch...",False,True,"{'old': 61731, 'new': 61919}","{'old': 1427172934, 'new': 1508596931}",True


In [31]:
data_2['target'] = data_2['bot'].astype(int)

In [32]:
data_2['n_digits_name'] = sum(c.isdigit() for c in data_2['user'])
data_2['digit_ratio_name'] = data_2['n_digits_name']/len(data_2['user'])
find_lead_digits = lambda name: len(re.findall('^\d+', name)[0]) if name[0].isdigit() else 0
data_2['lead_digits_name'] = data_2['user'].apply(find_lead_digits)
data_2['uniq_char_ratio_name'] = len(set(c.isalpha() for c in data_2['user']))/len(data_2['user'])

In [33]:
data_2.columns

Index(['Unnamed: 0', '$schema', 'meta', 'id', 'type', 'namespace', 'title',
       'comment', 'timestamp', 'user', 'log_id', 'log_type', 'log_action',
       'log_params', 'log_action_comment', 'server_url', 'server_name',
       'server_script_path', 'wiki', 'parsedcomment', 'minor', 'patrolled',
       'length', 'revision', 'bot', 'target', 'n_digits_name',
       'digit_ratio_name', 'lead_digits_name', 'uniq_char_ratio_name'],
      dtype='object')

In [34]:
data_2.head()

,Unnamed: 0,$schema,meta,id,type,namespace,title,comment,timestamp,user,log_id,log_type,log_action,log_params,log_action_comment,server_url,server_name,server_script_path,wiki,parsedcomment,minor,patrolled,length,revision,bot,target,n_digits_name,digit_ratio_name,lead_digits_name,uniq_char_ratio_name
0,0,/mediawiki/recentchange/1.0.0,{'uri': 'https://en.wikipedia.org/wiki/User:41...,1.430187e+09,log,2,User:41.155.9.82,{{blocked p2p proxy|ip=41.155.9.82|isp=Fixed-B...,1633526855,ST47ProxyBot,122426220.0,block,block,"{'duration': '71 hours', 'flags': 'nocreate', ...",blocked User:41.155.9.82 (account creation blo...,https://en.wikipedia.org,en.wikipedia.org,/w,enwiki,{{blocked p2p proxy|ip=41.155.9.82|isp=Fixed-B...,NaN,NaN,NaN,NaN,True,1,0,0.0,0,0.066667
1,1,/mediawiki/recentchange/1.0.0,{'uri': 'https://www.wikidata.org/wiki/Q174788...,1.556013e+09,edit,0,Q17478814,/* wbsetsitelink-add:1|commonswiki */ Category...,1633526854,Deanlaw,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org,www.wikidata.org,/w,wikidatawiki,"‎<span dir=""auto""><span class=""autocomment"">Ad...",False,True,"{'old': 18319, 'new': 18449}","{'old': 1447448374, 'new': 1508596930}",False,0,0,0.0,0,0.066667
2,2,/mediawiki/recentchange/1.0.0,{'uri': 'https://en.wikipedia.org/wiki/User:10...,1.430187e+09,log,2,User:103.108.61.85,{{blocked p2p proxy|ip=103.108.61.85|isp=INCOM...,1633526854,ST47ProxyBot,122426218.0,block,block,"{'duration': '71 hours', 'flags': 'nocreate', ...",blocked User:103.108.61.85 (account creation b...,https://en.wikipedia.org,en.wikipedia.org,/w,enwiki,{{blocked p2p proxy|ip=103.108.61.85|isp=INCOM...,NaN,NaN,NaN,NaN,True,1,0,0.0,0,0.066667
3,3,/mediawiki/recentchange/1.0.0,{'uri': 'https://www.wikidata.org/wiki/Q350193...,1.556013e+09,edit,0,Q3501934,/* wbsetdescription-add:1|yo */ agbábọ́ọ̀lù ọm...,1633526854,T Cells,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org,www.wikidata.org,/w,wikidatawiki,"‎<span dir=""auto""><span class=""autocomment"">Ad...",False,True,"{'old': 26326, 'new': 26436}","{'old': 1508596926, 'new': 1508596929}",False,0,0,0.0,0,0.066667
4,4,/mediawiki/recentchange/1.0.0,{'uri': 'https://www.wikidata.org/wiki/Q384910...,1.556013e+09,edit,0,Q38491095,/* wbeditentity-update-languages-short:0||nl *...,1633526854,Edoderoobot,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org,www.wikidata.org,/w,wikidatawiki,"‎<span dir=""auto""><span class=""autocomment"">Ch...",False,True,"{'old': 61731, 'new': 61919}","{'old': 1427172934, 'new': 1508596931}",True,1,0,0.0,0,0.066667


In [35]:
data_2.drop(['Unnamed: 0', '$schema', 'meta', 'id', 'type', 'namespace', 'title',
       'comment', 'timestamp', 'log_id', 'log_type', 'log_action',
       'log_params', 'log_action_comment', 'server_url', 'server_name',
       'server_script_path', 'wiki', 'parsedcomment', 'minor', 'patrolled',
       'length', 'revision', 'bot'], axis=1, inplace=True)

In [36]:
data_2.head()

,user,target,n_digits_name,digit_ratio_name,lead_digits_name,uniq_char_ratio_name
0,ST47ProxyBot,1,0,0.0,0,0.066667
1,Deanlaw,0,0,0.0,0,0.066667
2,ST47ProxyBot,1,0,0.0,0,0.066667
3,T Cells,0,0,0.0,0,0.066667
4,Edoderoobot,1,0,0.0,0,0.066667


In [72]:
test_size = 0.33 #should decide
X_train0, X_test0, y_train, y_test = train_test_split(data_2.drop(['target'], axis=1), data_2['target'], test_size=test_size, random_state=8)
X_train = X_train0.drop('user', axis=1)
X_test = X_test0.drop('user', axis=1)

In [41]:
X_train0.head()

,user,n_digits_name,digit_ratio_name,lead_digits_name,uniq_char_ratio_name
23,ST47ProxyBot,0,0.0,0,0.066667
11,No Great Shaker,0,0.0,0,0.066667
2,ST47ProxyBot,0,0.0,0,0.066667
14,T Cells,0,0.0,0,0.066667
21,RottenBot,0,0.0,0,0.066667


# Classifiers

#### RandomForest

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [43]:
model_rf = RandomForestClassifier()

params_rf = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [100, 200, 400, 600, 800],
    }

#### XGBoost

In [44]:
import xgboost as xgb

In [45]:
model_xgb = xgb.XGBClassifier()

params_xgb = {
    'model__max_depth': [2, 3, 5, 7, 10],
    'model__n_estimators': [10, 100, 500],
}

#### Logistic Regression

In [65]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

params_lr = {
    
}

# GridSearch

In [46]:
np.random.seed(1111)

In [47]:
model = model_xgb
params_grid = params_xgb

In [48]:
grid = GridSearchCV(estimator=model, param_grid=params_grid, cv= 5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'model__max_depth': [2, 3, 5, 7, 10],
                         'model__n_estimators': [10, 100, 500]},
             pre_dispatch='2*n_jobs', refit=True,

In [49]:
grid.best_params_

{'model__max_depth': 2, 'model__n_estimators': 10}

In [50]:
y_pred = grid.best_estimator_.predict(X_test)

In [51]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
y_test

25    0
0     1
7     0
18    1
15    1
4     1
29    0
1     0
22    0
6     1
Name: target, dtype: int64

In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [43]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")

Accuracy: 0.5
Precision: 0.0
Recall: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Getting bad emails/names/whatever

In [56]:
bot_idx = [i for i, e in enumerate(y_test) if e != 0]

In [58]:
bot_idx

[1, 3, 4, 5, 9]

In [76]:
X_test_user = pd.DataFrame(X_test0['user']).reset_index().drop('index', axis=1)

In [80]:
X_test_user

,user
0,T Cells
1,ST47ProxyBot
2,Repf72
3,SchlurcherBot
4,ST47ProxyBot
5,Edoderoobot
6,WindEwriX
7,Deanlaw
8,SomeRandomPasserby
9,Soweego bot


In [79]:
X_test_user.loc[bot_idx]

,user
1,ST47ProxyBot
3,SchlurcherBot
4,ST47ProxyBot
5,Edoderoobot
9,Soweego bot
